##  MiniFlow

### Introduction 

A neural network is a graph of mathematical funstions such as linear combinations and activation functions. The graph consists of nodes, and edges.

Nodes in each layer(except for the input layer) perform mathematical functions using inputs from nodes in the previous layers. For example, a node could represent $f(x,y) = x + y$, where $x$ and $y$ are input values from nodes in the previous layer.

The edges in the graph describe the connection between the nodes, along which the values flow from one layer to the next.

MiniFlow is a neural network library meant to be a miniature version of Google's TensorFlow library. The library implements backpropogation and forward passing using a simple sigmoid activation function. 


In [89]:
import numpy as np
import random

### The Node class

In [90]:
class Node():
    def __init__(self, inbound_nodes=[]):
        # Node(s) from which this Node receives values
        self.inbound_nodes = inbound_nodes
        
        # Node(s) to which this Node passes values
        self.outbound_nodes = []
        
        # For each inbound Node here, add this Node as an outbound Node
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
        # A calculated value
        self.value = None
        
    def forward(self):
        """
        Forword propagation.
        
        Compute the output value based on 'inbound_nodes' and 
        store the result in self.value.
        """
        raise NotImplemented
        


### Input node class

Input nodes do no calculations but only hold values to be passed forward

In [91]:
class Input(Node):
    def __init__(self):
        # An Input node has no inbound nodes
        Node.__init__(self)
        
    def forward(self, value=None):
        # Overwrite the value if one is passed in
        if value is not None:
            self.value = value
            

### Add node class

Takes 2 inbound nodes, x, and y, and adds the values of those nodes.

In [92]:
class Add(Node):
    def __init__(self, *args):
        Node.__init__(self, args)
    
    def forward(self):
        """
        Calculates sum and passes it forward
        """
        self.value = sum([node.value for node in self.inbound_nodes])

### Function to run a topological sort on a list of nodes

This function performs a topological sort based on Khan's algorithm.

In [93]:
def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` node and the value is the respective value feed to that node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L

### Forward pass

Perform forward pass on sorted nodes

In [94]:
def forward_pass(output_node, sorted_nodes):
    """
    Performs a forward pass through a list of sorted nodes.

    Arguments:

        `output_node`: A node in the graph, should be the output node (have no outgoing edges).
        `sorted_nodes`: A topologically sorted list of nodes.

    Returns the output Node's value
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.value

### Test addition with varying number of input nodes

In [95]:
"""
This script builds and runs a graph with miniflow.

There is no need to change anything to solve this quiz!

However, feel free to play with the network! Can you also
build a network that solves the equation below?

(x + y) + y
"""

x, y = Input(), Input()

f = Add(x, y)

feed_dict = {x: 10, y: 5}

sorted_nodes = topological_sort(feed_dict)
output = forward_pass(f, sorted_nodes)

# NOTE: because topological_sort set the values for the `Input` nodes we could also access
# the value for x with x.value (same goes for y).
print("{} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], output))


10 + 5 = 15 (according to miniflow)


In [96]:
"""
No need to change anything here!

If all goes well, this should work after you
modify the Add class in miniflow.py.
"""

x, y, z = Input(), Input(), Input()

f = Add(x, y, z)

feed_dict = {x: 4, y: 5, z: 10}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

# should output 19
print("{} + {} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], feed_dict[z], output))


4 + 5 + 10 = 19 (according to miniflow)


### Multiplication node class

In [97]:
class Mul(Node):
    def __init__(self, *args):
        Node.__init__(self, args)
    
    def forward(self):
        """
        Calculates product and passes it forward
        """
        self.value = 1
        for node in self.inbound_nodes:
            self.value*=node.value 

###  Test multiplication with varying number of nodes

In [98]:
x, y = Input(), Input()

f = Mul(x, y)

feed_dict = {x: 10, y: 5}

sorted_nodes = topological_sort(feed_dict)
output = forward_pass(f, sorted_nodes)

# NOTE: because topological_sort set the values for the `Input` nodes we could also access
# the value for x with x.value (same goes for y).
print("{} * {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], output))


10 * 5 = 50 (according to miniflow)


In [99]:
x, y, z = Input(), Input(), Input()

f = Mul(x, y, z)

feed_dict = {x: 4, y: 5, z: 10}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

# should output 200
print("{} * {} * {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], feed_dict[z], output))


4 * 5 * 10 = 200 (according to miniflow)


Consider the following equation

$$\sigma = \sum_{i} x_iw_i + b$$

inputs, x (vector)  
weights, w (vector)  
bias, b (scalar)

See below for a node/neuron that applies the above linear equation

### Linear Node class

In [100]:
class Linear(Node):
    def __init__(self, X, W, b):
        Node.__init__(self, [X, W, b])

        # NOTE: The weights and bias properties here are not
        # numbers, but rather references to other nodes.
        # The weight and bias values are stored within the
        # respective nodes.

    def forward(self):
        """
        Set self.value to the value of the linear function output.
        """
        from numpy import dot
        X = self.inbound_nodes[0].value # inputs
        W = self.inbound_nodes[1].value # weights
        assert len(X) == len(W),"Inconsistent number of inputs and weights"
        b = self.inbound_nodes[2].value # bias
        self.value = dot(X, W) + b # Z = XW + b

### Test Linear node with matrices

In [101]:
import numpy as np

X, W, b = Input(), Input(), Input()

f = Linear(X, W, b)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

"""
Output should be:
[[-9., 4.],
[-9., 4.]]
"""
print(output)


[[-9.  4.]
 [-9.  4.]]


### Sigmoid node

The sigmoid function is defined as follows

$$\sigma(x) = \frac{1}{1+e^{-x}}$$

The sigmoid node will take one input and apply the above function during a forward pass, passing any result to the next layer, if there is one

In [102]:
class Sigmoid(Node):
    """
    You need to fix the `_sigmoid` and `forward` methods.
    """
    def __init__(self, node):
        Node.__init__(self, [node])

    def _sigmoid(self, x):
        """
        This method is separate from `forward` because it
        will be used with `backward` as well.

        `x`: A numpy array-like object.
        """
        return 1. / (1. + np.exp(-x))
    
    def forward(self):
        """
        Set the value of this node to the result of the
        sigmoid function.
        """
        input_value = self.inbound_nodes[0].value
        self.value = self._sigmoid(input_value)

In [103]:
X, W, b = Input(), Input(), Input()

f = Linear(X, W, b)
g = Sigmoid(f)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(g, graph)

"""
Output should be:
[[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]
"""
print(output)

[[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]


### Calculating loss/cost

We need a way to determine how correct the output of our network is, we can calculate this using the mean squared error (MSE). Like so:

$$C(w,b) = \frac{1}{m}\sum_{x}\vert\vert y(x) - a\vert\vert^2$$

We shall implement this in the forward pass of it's own node

In [104]:
class MSE(Node):
    def __init__(self, y, a):
        """
        The mean squared error cost function.
        Should be used as the last node for a network.
        """
        # Call the base class' constructor.
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.
        #
        # For example, if we subtract an array of shape (3,) from an array of shape
        # (3,1) we get an array of shape(3,3) as the result when we want
        # an array of shape (3,1) instead.
        #
        # Making both arrays (3,1) insures the result is (3,1) and does
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)
        m = self.inbound_nodes[0].value.shape[0]
        self.value = np.mean(np.square(y-a))

### Test MSE

In [105]:
y, a = Input(), Input()
cost = MSE(y, a)

y_ = np.array([1, 2, 3])
a_ = np.array([4.5, 5, 10])

feed_dict = {y: y_, a: a_}
graph = topological_sort(feed_dict)
# forward pass
forward_pass(cost, graph)

"""
Expected output

23.4166666667
"""
print(cost.value)

23.4166666667


### Gradient Descent

In [106]:
def gradient_descent_update(x, gradx, learning_rate):
    """
    Performs a gradient descent update.
    """
    x -= (learning_rate * gradx)
    return x

In [107]:
def f(x):
    """
    Quadratic function.

    It's easy to see the minimum value of the function
    is 5 when is x=0.
    """
    return x**2 + 5

In [108]:
def df(x):
    """
    Derivative of `f` with respect to `x`.
    """
    return 2*x

In [109]:
x = random.randint(0, 10000)
# TODO: Set the learning rate
learning_rate = 0.001
epochs = 100

In [110]:
for i in range(epochs+1):
    cost = f(x)
    gradx = df(x)
    print("EPOCH {}: Cost = {:.3f}, x = {:.3f}".format(i, cost, gradx))
    x = gradient_descent_update(x, gradx, learning_rate)

EPOCH 0: Cost = 5053509.000, x = 4496.000
EPOCH 1: Cost = 5033315.198, x = 4487.008
EPOCH 2: Cost = 5013202.090, x = 4478.034
EPOCH 3: Cost = 4993169.355, x = 4469.078
EPOCH 4: Cost = 4973216.670, x = 4460.140
EPOCH 5: Cost = 4953343.716, x = 4451.219
EPOCH 6: Cost = 4933550.175, x = 4442.317
EPOCH 7: Cost = 4913835.728, x = 4433.432
EPOCH 8: Cost = 4894200.061, x = 4424.566
EPOCH 9: Cost = 4874642.857, x = 4415.716
EPOCH 10: Cost = 4855163.804, x = 4406.885
EPOCH 11: Cost = 4835762.590, x = 4398.071
EPOCH 12: Cost = 4816438.902, x = 4389.275
EPOCH 13: Cost = 4797192.433, x = 4380.497
EPOCH 14: Cost = 4778022.872, x = 4371.736
EPOCH 15: Cost = 4758929.912, x = 4362.992
EPOCH 16: Cost = 4739913.248, x = 4354.266
EPOCH 17: Cost = 4720972.575, x = 4345.558
EPOCH 18: Cost = 4702107.588, x = 4336.866
EPOCH 19: Cost = 4683317.986, x = 4328.193
EPOCH 20: Cost = 4664603.468, x = 4319.536
EPOCH 21: Cost = 4645963.732, x = 4310.897
EPOCH 22: Cost = 4627398.481, x = 4302.275
EPOCH 23: Cost = 4608

### Backpropogation

Backpropogation is the mechanism by which we go backwards through the network calculating the deritives of the node and using that derivative to improve our weights and biases(i.e minimize cost/loss throughout the network).

Basically in order to figure out how we should alter a parameter to minimize the cost, we must first find out what effect that parameter has on the cost.

The gradient takes into account the effect each parameter has on the cost, so that's how we find the direction of steepest ascent. Backpropogation is just a clever application of the chain rule as we aim to calculate the total change a parameter has on the cost

### Updating the Node classes

We will now update the Node class to have a backward function

In [117]:
class Node():
    def __init__(self, inbound_nodes=[]):
        # Node(s) from which this Node receives values
        self.inbound_nodes = inbound_nodes
        
        # Node(s) to which this Node passes values
        self.outbound_nodes = []
        
        # For each inbound Node here, add this Node as an outbound Node
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
        # A calculated value
        self.value = None
        
        # New property! Keys are the inputs to this node and
        # their values are the partials of this node with
        # respect to that input.
        self.gradients = {}
        
    def forward(self):
        """
        Forword propagation.
        
        Compute the output value based on 'inbound_nodes' and 
        store the result in self.value.
        """
        raise NotImplemented
    
    def backward(self):
        """
        Every node that uses this class as a base class will need to define 
        its own `backward` method.
        """
        raise NotImplemented

In [114]:
class Input(Node):
    """
    A generic input into the network.
    """
    def __init__(self):
        # The base class constructor has to run to set all
        # the properties here.
        #
        # The most important property on an Input is value.
        # self.value is set during `topological_sort` later.
        Node.__init__(self)

    def forward(self):
        # Do nothing because nothing is calculated.
        pass

    def backward(self):
        # An Input node has no inputs so the gradient (derivative)
        # is zero.
        # The key, `self`, is reference to this object.
        self.gradients = {self: 0}
        # Weights and bias may be inputs, so you need to sum
        # the gradient from output gradients.
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]
            self.gradients[self] += grad_cost * 1

In [115]:
class Linear(Node):
    """
    Represents a node that performs a linear transform.
    """
    def __init__(self, X, W, b):
        # The base class (Node) constructor. Weights and bias
        # are treated like inbound nodes.
        Node.__init__(self, [X, W, b])

    def forward(self):
        """
        Performs the math behind a linear transform.
        """
        X = self.inbound_nodes[0].value
        W = self.inbound_nodes[1].value
        b = self.inbound_nodes[2].value
        self.value = np.dot(X, W) + b

    def backward(self):
        """
        Calculates the gradient based on the output values.
        """
        # Initialize a partial for each of the inbound_nodes.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}
        # Cycle through the outputs. The gradient will change depending
        # on each output, so the gradients are summed over all outputs.
        for n in self.outbound_nodes:
            # Get the partial of the cost with respect to this node.
            grad_cost = n.gradients[self]
            # Set the partial of the loss with respect to this node's inputs.
            self.gradients[self.inbound_nodes[0]] += np.dot(grad_cost, self.inbound_nodes[1].value.T)
            # Set the partial of the loss with respect to this node's weights.
            self.gradients[self.inbound_nodes[1]] += np.dot(self.inbound_nodes[0].value.T, grad_cost)
            # Set the partial of the loss with respect to this node's bias.
            self.gradients[self.inbound_nodes[2]] += np.sum(grad_cost, axis=0, keepdims=False)

In [116]:
class MSE(Node):
    def __init__(self, y, a):
        """
        The mean squared error cost function.
        Should be used as the last node for a network.
        """
        # Call the base class' constructor.
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.
        #
        # For example, if we subtract an array of shape (3,) from an array of shape
        # (3,1) we get an array of shape(3,3) as the result when we want
        # an array of shape (3,1) instead.
        #
        # Making both arrays (3,1) ensures the result is (3,1) and does
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)

        self.m = self.inbound_nodes[0].value.shape[0]
        # Save the computed output for backward.
        self.diff = y - a
        self.value = np.mean(self.diff**2)

    def backward(self):
        """
        Calculates the gradient of the cost.

        This is the final node of the network so outbound nodes
        are not a concern.
        """
        self.gradients[self.inbound_nodes[0]] = (2 / self.m) * self.diff
        self.gradients[self.inbound_nodes[1]] = (-2 / self.m) * self.diff

### Updating the forward_pass function

In [113]:
def forward_and_backward(graph):
    """
    Performs a forward pass and a backward pass through a list of sorted nodes.

    Arguments:

        `graph`: The result of calling `topological_sort`.
    """
    # Forward pass
    for n in graph:
        n.forward()

    # Backward pass
    # see: https://docs.python.org/2.3/whatsnew/section-slices.html
    for n in graph[::-1]:
        n.backward()

### Updating the Sigmoid node to implement backward pass

Recall 

$$\sigma(x) = \frac{1}{1+e^{-x}}$$

Hence it can be proven that

$$\frac{\partial \sigma(x)}{\partial x} = \sigma(x)*(1 - \sigma(x))$$

So we need to implement the above equation in our backward pass for our sigmoid node

In [121]:
class Sigmoid(Node):
    """
    Represents a node that performs the sigmoid activation function.
    """
    def __init__(self, node):
        # The base class constructor.
        Node.__init__(self, [node])

    def _sigmoid(self, x):
        """
        This method is separate from `forward` because it
        will be used with `backward` as well.

        `x`: A numpy array-like object.
        """
        return 1. / (1. + np.exp(-x))

    def forward(self):
        """
        Perform the sigmoid function and set the value.
        """
        input_value = self.inbound_nodes[0].value
        self.value = self._sigmoid(input_value)

    def backward(self):
        """
        Calculates the gradient using the derivative of
        the sigmoid function.
        """
        # Initialize the gradients to 0.
        self.gradients = {n: np.zeros_like(n.value) for n in self.inbound_nodes}

        # Cycle through the outputs. The gradient will change depending
        # on each output, so the gradients are summed over all outputs.
        for n in self.outbound_nodes:
            # Get the partial of the cost with respect to this node.
            grad_cost = n.gradients[self]
            sigmoid = self.value
            x = self.inbound_nodes[0].value
            self.gradients[self.inbound_nodes[0]] += grad_cost * sigmoid * (1 - sigmoid) 

### Test new Sigmoid Node

In [122]:
X, W, b = Input(), Input(), Input()
y = Input()
f = Linear(X, W, b)
a = Sigmoid(f)
cost = MSE(y, a)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2.], [3.]])
b_ = np.array([-3.])
y_ = np.array([1, 2])

feed_dict = {
    X: X_,
    y: y_,
    W: W_,
    b: b_,
}

graph = topological_sort(feed_dict)
forward_and_backward(graph)
# return the gradients for each Input
gradients = [t.gradients[t] for t in [X, y, W, b]]

"""
Expected output

[array([[ -3.34017280e-05,  -5.01025919e-05],
       [ -6.68040138e-05,  -1.00206021e-04]]), array([[ 0.9999833],
       [ 1.9999833]]), array([[  5.01028709e-05],
       [  1.00205742e-04]]), array([ -5.01028709e-05])]
"""
print(gradients)

[array([[ -3.34017280e-05,  -5.01025919e-05],
       [ -6.68040138e-05,  -1.00206021e-04]]), array([[ 0.9999833],
       [ 1.9999833]]), array([[  5.01028709e-05],
       [  1.00205742e-04]]), array([ -5.01028709e-05])]


### Stochastic Gradient Update

Stochastic Gradient Descent (SGD) is a version of Gradient Descent where on each forward pass a batch of data is randomly sampled from total dataset. Remember when we talked about the batch size earlier? That's the size of the batch. Ideally, the entire dataset would be fed into the neural network on each forward pass, but in practice, it's not practical due to memory constraints. SGD is an approximation of Gradient Descent, the more batches processed by the neural network, the better the approximation.

A naïve implementation of SGD involves:

* Randomly sample a batch of data from the total dataset.
* Running the network forward and backward to calculate the gradient (with data from (1)).
* Apply the gradient descent update.
* Repeat steps 1-3 until convergence or the loop is stopped by another mechanism (i.e. the number of epochs).

If all goes well, the network's loss should generally trend downwards, indicating more useful weights and biases over time.

In [123]:
def sgd_update(trainables, learning_rate=1e-2):
    """
    Updates the value of each trainable with SGD.

    Arguments:

        `trainables`: A list of `Input` Nodes representing weights/biases.
        `learning_rate`: The learning rate.
    """
    # TODO: update all the `trainables` with SGD
    # You can access and assign the value of a trainable with `value` attribute.
    # Example:
    for t in trainables:
        partial = t.gradients[t]
        t.value = t.value - learning_rate * partial